In [16]:
import pandas as pd

### iSEAR preprocessing

In [17]:
df = pd.read_csv("./dataset/original_datasets/isear.csv", sep="|")

In [18]:
df = df[["Field1", "SIT"]]
df = df.rename(columns={"Field1": "emotion", "SIT": "comment"})

In [19]:
df = df.drop(df[df.comment.str.startswith("[") & df.comment.str.endswith("]")].index)

In [20]:
df = df.drop_duplicates(subset="comment")

In [21]:
df['emotion'].value_counts()

fear       1073
anger      1070
joy        1066
disgust    1063
guilt      1045
sadness    1040
shame      1040
Name: emotion, dtype: int64

In [22]:
df = df.drop(df[df.emotion == "shame"].index)
df = df.drop(df[df.emotion == "guilt"].index)

In [23]:
from spellchecker import SpellChecker

spell = SpellChecker()

def spell_check(x):
    correct_word = []
    mispelled_word = x.split()
    for word in mispelled_word:
        correct_word.append(spell.correction(word))
    return ' '.join(correct_word)

In [24]:
df['comment'] = df['comment'].apply(lambda x: spell_check(x))

In [25]:
df

,emotion,comment
0,joy,During the period of falling in love each time...
1,fear,When I was involved in a traffic accident
2,anger,When I was driving home after several days of ...
3,sadness,When I lost the person who meant the most to me
4,disgust,The time I knocked a deer down - the sight of ...
...,...,...
7659,joy,I received a letter from a distant friend
7660,fear,My parents were out and I was the eldest at ho...
7661,anger,Two years back someone invited me to be the tu...
7662,sadness,I had taken the responsibility to do something...


In [41]:
df = df.drop(df[df['comment'].str.split(" ").apply(len) < 5].index)

In [42]:
df.describe()

,emotion,comment
count,5149,5149
unique,5,5149
top,anger,During the period of falling in love each time...
freq,1043,1


In [43]:
df['emotion'].value_counts()

anger      1043
fear       1042
joy        1040
disgust    1039
sadness     985
Name: emotion, dtype: int64

In [44]:
import re

def remove_non_utf_8_chars_from_text(text):
    pattern = re.compile('(?i)[^a-z0-9 \n]+')
    return pattern.sub('', text)

In [46]:
df['comment'] = df['comment'].apply(remove_non_utf_8_chars_from_text)

In [47]:
df

,emotion,comment
0,joy,During the period of falling in love each time...
1,fear,When I was involved in a traffic accident
2,anger,When I was driving home after several days of ...
3,sadness,When I lost the person who meant the most to me
4,disgust,The time I knocked a deer down the sight of t...
...,...,...
7659,joy,I received a letter from a distant friend
7660,fear,My parents were out and I was the eldest at ho...
7661,anger,Two years back someone invited me to be the tu...
7662,sadness,I had taken the responsibility to do something...
